In [2]:
import cyvcf2
import numpy as np
import tsinfer
import tskit
from tskit import MISSING_DATA
import pandas as pd
import pyslim
import allel
from tqdm import tqdm

## files 
greneNet_final_v1.1.recode.vcf: og vcf file

wholegenome_offset.vcf : same vcf file but with the positions values changed with the awk command

wholegenome_offset_samplefile: the result of converting the vcf file into a samplefile object (the one needed to infer the tree with tsinfer)

wholegenome_offset.trees: the tree after inference keeping procesesd = False

wholegenome_offset_processed.trees: the tree after running process but keeping flanking sites 

wholegenome_offset_baselinetree.trees: tree with all the structure of the metadata to be taken by slim but with all the mutations. this tree is the baseline to eliminate mutations based on teh different genetic architectures 

mapper_realid_metadataid_wholegenome: the mapper from our id nodes to the og tree id nodes 

output_completegenome.vcf : the result of the simulation with addded mutations

output_completegenome_chr.vcf: the results with corrected chromosomes positions 

bcftools view greneNet_final_v1.1.recode.vcf.gz --regions 1 -o chr1.vcf

chr1.vcf  : onyl chromosme 1 to test 

two_samples : is a. vcf file with only 2 samples, and chr from 

bcftools view -s 159,265 -r 1:1-3042767 greneNet_final_v1.1.recode.vcf.gz > two_samples.vcf

greneNet_final_v1.1_nohet.recode.vcf : no heterozygous sites

wholegenome_offset_nohet.vcf no heterozygous sites and the posiitons shifted

wholegenome_offset_nohet_samplefile : the sample file

## Part 0: Modifications in vcf file
Becuase tsinfer is builed to get one chromosome, we will need to provide vcf file with no overlapping positions, this means that the positions column in the vcf file should go from 0 to 119146348. For that:

In [71]:
awk 'BEGIN {OFS="\t"} /^#/ {print} !/^#/   
                {if ($1 == "2") $2 = $2 + 30427671; 
                else if ($1 == "3") $2 = $2 + 50125960;    
                else if ($1 == "4") $2 = $2 + 73585790; 
                else if ($1 == "5") $2 = $2 + 92170846;        
                print}' greneNet_final_v1.1_nohet.recode.vcf > wholegenome_offset_nohet.vcf

SyntaxError: EOL while scanning string literal (2066762022.py, line 1)

## First part: runs only once 
Import VCF -> create samplefile -> estimate tree seq -> annotate tree seq so is readabale by SLiM + create a mapper of nodes so mutations can be overlaied in the future

In [ ]:
## define chromosome length (for whole genome is 119146348)
##contig=<ID=Chr1,length=30427671>
##contig=<ID=Chr2,length=19698289>
##contig=<ID=Chr3,length=23459830>
##contig=<ID=Chr4,length=18585056>
##contig=<ID=Chr5,length=26975502>
chromosome_length = 30427671# 119146348  ## thsi is for the two samples 3042767

In [ ]:
def add_populations(vcf, samples):
    """
    Add tsinfer Population objects and returns a list of IDs corresponding to the VCF samples.
    """
    # this function in my case just creates a list of 0 indicating the pop id, for 
    # the x amount of samples the vcf file has 
    
    pop_lookup = {}
    pop_lookup["0"] = samples.add_population()
    population_belong = []
    for sample in vcf.samples:
        population_belong.append(0)
    
    return population_belong

In [72]:
def add_diploid_individuals(vcf, samples, populations):
    ## this function just iterate through the samples (individual) names and then assign 
    ## them to the populations and most importatntly states that they are diploid individuals 
    for name, population in zip(vcf.samples, populations):
        samples.add_individual(ploidy=2, metadata={"name": name}, population=population)

In [73]:
def add_diploid_sites(vcf, samples):
    """
    Read the sites in the vcf and add them to the samples object.
    """
    # You may want to change the following line, e.g. here we allow
    # "*" (a spanning deletion) to be a valid allele state
    allele_chars = set("ATGCatgc*")
    pos = 0
    progressbar = tqdm(total=30427671, desc="Read VCF", unit='bp')

    #progressbar = tqdm(total=samples.sequence_length, desc="Read VCF", unit='bp')
    for variant in vcf:  # Loop over variants, each assumed at a unique site
        progressbar.update(variant.POS - pos)

        ## each variant in the cyvcf2 library is just one line in the vcf file 
        ## ignoring of course the info lines
        #progressbar.update(variant.POS - pos)
        
        ## access the position of the variant 
        if pos == variant.POS:
            print(f"Duplicate entries at position {pos}, ignoring all but the first")
            continue
        else:
            pos = variant.POS
        #print(pos)
        ## check that the variants are phased 
        if any([not phased for _, _, phased in variant.genotypes]):
            raise ValueError("Unphased genotypes for variant at position", pos)
        alleles = [variant.REF.upper()] + [v.upper() for v in variant.ALT]
        
        # in my case the ancestral allele is always the ref 
        ancestral = variant.REF.upper()
        if ancestral == "." or ancestral == "":
            ancestral_allele = MISSING_DATA
            # alternatively, you could specify `ancestral = variant.REF.upper()`
        else:
            ancestral_allele = alleles.index(ancestral)
        # Check we have ATCG alleles
        for a in alleles:
            if len(set(a) - allele_chars) > 0:
                print(f"Ignoring site at pos {pos}: allele {a} not in {allele_chars}")
                continue

        # Map original allele indexes to their indexes in the new alleles list.
        genotypes = [g for row in variant.genotypes for g in row[0:2]]

        ## now actually adding the info based on the thigns retrieved before
        samples.add_site(pos, genotypes, alleles, ancestral_allele=ancestral_allele)

### Import VCF

In [74]:
vcf_location = "qtl2_contrib_correct.vcf"

In [75]:
chromosome_name = vcf_location.split('.')[0]

In [76]:
chromosome_name

'qtl2_contrib_correct'

In [77]:
## read the vcf file with the cyvcf2 library 
vcf = cyvcf2.VCF(vcf_location)

In [78]:
with tsinfer.SampleData(
    path=chromosome_name +"_samplefile", sequence_length=chromosome_length) as samples:
    ## add dipplod individuals and populations
    populations = add_populations(vcf, samples)
    add_diploid_individuals(vcf, samples, populations)
    #####
    add_diploid_sites(vcf, samples)

print(
    "Sample file created for {} samples ".format(samples.num_samples)
    + "({} individuals) ".format(samples.num_individuals)
    + "with {} variable sites.".format(samples.num_sites),
    flush=True,
)

Read VCF:  76%|███████▋  | 23223457/30427671 [00:00<00:00, 57203771.32bp/s] 

Sample file created for 450 samples (225 individuals) with 652 variable sites.


In [12]:
#samples = tsinfer.load('wholegenome_offset_subset_samplefile')

### Infer tree seq 

python tree_infer_noprocess.py wholegenome_offset_nohet_samplefile wholegenome_offset_nohet.trees
python tree_infer.py chr1_samplefile chr1.trees

In [3]:
ts = tsinfer.infer(samples,num_threads=10)

NameError: name 'samples' is not defined

In [80]:
ts

### Annotate treeseq 

In [15]:
#ts = tskit.load('../treeseq/wholegenome_offset.trees')

#ts_processed = tsinfer.post_process(ts, erase_flanks=False) 

#ts_processed.dump('../treeseq/wholegenome_offset_processed.trees')

In [6]:
ts = tskit.load('qtl2_contrib.trees')

In [7]:
ts

In [8]:
tables = ts.dump_tables()

In [9]:
## modify metadata to make it compatible with slim
tables.metadata_schema = pyslim.slim_metadata_schemas['tree_sequence']

pyslim.annotate_tables(tables, model_type="nonWF", tick=1,annotate_mutations=False)

### Create mapper for nodes

In [10]:
## change nodes id in metadata to remember them and create mapper_realid_metadataid which will serve to map mutations back in their nodes
tables.nodes.clear()

counter_for_samples = 0 
counter_for_ancestral = -1

for n in ts.nodes():
    if n.flags == 1:
        tables.nodes.append(n.replace(metadata={'slim_id': counter_for_samples, 'is_null': False, 'genome_type': 0}))
        counter_for_samples += 1
    else:
        tables.nodes.append(n.replace(metadata={'slim_id': counter_for_ancestral, 'is_null': True, 'genome_type': 0}))
        counter_for_ancestral -= 1

In [11]:
my_ids=[]
for i in tables.nodes:
    my_ids.append(i.metadata['slim_id'])

mapper_realid_metadataid = pd.DataFrame({'real_id':range(0, len(tables.nodes)), 'my_ids_metadata':my_ids})

In [12]:
mapper_realid_metadataid

,real_id,my_ids_metadata
0,0,0
1,1,1
2,2,2
3,3,3
4,4,4
...,...,...
1355,1355,-906
1356,1356,-907
1357,1357,-908
1358,1358,-909


In [13]:
mapper_realid_metadataid.to_csv('mapper_realid_metadataid_short_tree_4test.csv')

### Save common tree for all simulations 

In [14]:
new_ts = tables.tree_sequence()

In [15]:
new_ts.dump('short_tree_4test.trees')

## Part 2 : specific to each trait architecture

Delete all the sites and mutations that are not causal -> save tree ready for SLiM

In [16]:
selected_sites = [16786062., 17435486., 18722427., 19304820., 19535796., 20546789.,
       21204718., 21956162., 22302386., 22575472.]

sc = [-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, 0.5, 0.5, 0.5]

pos_sc = pd.DataFrame({'pos': selected_sites, 'sc': sc})


tables = new_ts.dump_tables()

### Delete all the sites and mutations that are not causal 

In [17]:
complete_sites = pd.Series(tables.sites.position)

complete_sites.name = 'complete_sites'

mask_delete_sites = pd.merge(complete_sites, pos_sc['pos'], left_on = 'complete_sites', right_on = 'pos', how = 'left')['pos'].notna()

tables.sites.replace_with(tables.sites[mask_delete_sites])

tables.mutations.replace_with(tables.mutations[mask_delete_sites])

tables.mutations.site = np.array(range(0, len(tables.mutations))).astype('int32')

new_ts = tables.tree_sequence()

pos_table = pd.Series(tables.sites.position).reset_index()

right_order_pos = pos_sc.merge(pos_table, left_on='pos',right_on =0 ).sort_values('index')

new_ts = tables.tree_sequence()



In [18]:
new_ts

In [19]:
tables = new_ts.dump_tables()

In [20]:


tables.sites.clear()
for s in new_ts.sites():
    tables.sites.append(s.replace(ancestral_state=""))

tables.mutations.clear()
for k, (m, sc) in enumerate(zip(new_ts.mutations(), right_order_pos['sc'])):
    mm = pyslim.default_slim_metadata('mutation_list_entry')
    mm['selection_coeff'] = sc
    tables.mutations.append(
        m.replace(derived_state=str(k), metadata={'mutation_list': [mm]})
    )

In [21]:
## aca trasnforma todos los nodes a pertenecientes a la pobalcion 0 
#tables.nodes.clear()
#for n in new_ts.nodes():
#    tables.nodes.append(n.replace(population=0))

In [22]:
## testing to delete all mutations 

In [23]:
new_ts

In [24]:
tables = ts.dump_tables()

In [25]:
tables.mutations

id,site,node,time,derived_state,parent,metadata
0,0,572,nan,0,-1,{'mutation_list': [{'mutation_type': ...
1,1,1095,nan,1,-1,{'mutation_list': [{'mutation_type': ...
2,2,768,nan,2,-1,{'mutation_list': [{'mutation_type': ...
3,3,707,nan,3,-1,{'mutation_list': [{'mutation_type': ...
4,4,822,nan,4,-1,{'mutation_list': [{'mutation_type': ...
5,5,635,nan,5,-1,{'mutation_list': [{'mutation_type': ...
6,6,868,nan,6,-1,{'mutation_list': [{'mutation_type': ...
7,7,565,nan,7,-1,{'mutation_list': [{'mutation_type': ...
8,8,876,nan,8,-1,{'mutation_list': [{'mutation_type': ...
9,9,671,nan,9,-1,{'mutation_list': [{'mutation_type': ...


In [26]:
tables.sites.clear()
tables.mutations.clear()

In [27]:
new_tsm

NameError: name 'new_tsm' is not defined

In [28]:
tables.sites.clear()
for s in new_tsm.sites():
    tables.sites.append(s.replace(ancestral_state=""))

tables.mutations.clear()
for k, m in enumerate(new_tsm.mutations()):
    mm = pyslim.default_slim_metadata('mutation_list_entry')
    tables.mutations.append(
        m.replace(derived_state=str(k), metadata={'mutation_list': [mm]})
    )

NameError: name 'new_tsm' is not defined

In [29]:
new_tsm

NameError: name 'new_tsm' is not defined

In [108]:
tablesn = new_tsm.dump_tables()

In [111]:
tablesn.sites

id,position,ancestral_state,metadata


In [101]:
new_tsm = tables.tree_sequence()

In [102]:
new_ts.dump('testing_nomutationstree.trees')

### Save tree ready for SLiM

In [67]:
new_ts = tables.tree_sequence()

new_ts.dump('qtl2_contrib.trees')


In [68]:
new_ts

## Part3: Run simulationS SLiM

## Part 4: Overlay deleted mutations 

In [1]:
## import the new and the old tree
ts_new = tskit.load("test_output_tree.trees")
ts = tskit.load("common_tree_whole_genome.trees")

NameError: name 'tskit' is not defined

In [318]:
## extract surviving ndoes and comapre them to our old ndoes to place mtuations in the right place
surviving_nodes = []
for i in ts_new.tables.nodes:
    surviving_nodes.append(i.metadata['slim_id'])

In [319]:
## new nodes id and the ids i gave them in the past
new_mapper = pd.DataFrame({'new_ids': range(0, len(ts_new.tables.nodes)), 'my_ids_metadata':surviving_nodes})

In [320]:
## map old nodes with new nodes
mapper_lost_nodes = new_mapper.merge(mapper_realid_metadataid, left_on = 'my_ids_metadata', right_on = 'my_ids_metadata', how= 'right')

In [322]:
## create a mask to only keep from the old nodes the ones that survived the simulation
mask = mapper_lost_nodes['new_ids'].notna()

In [323]:
tables_og = ts.dump_tables()

In [324]:
## now filter old tables only based on surviving nodes 
tables_og.nodes.replace_with(tables_og.nodes[mask])

In [326]:
## now filter mutation table based on the surviving nodes, for that, extract the nodes 
old_nodes = tables_og.mutations.node

In [327]:
old_nodes = pd.Series(old_nodes)

In [328]:
old_nodes.name = 'old_nodes'

In [329]:
## create a dataframe relating the new and old nodes
replace_oldbynew_nodes = pd.merge(old_nodes, mapper_lost_nodes, left_on ='old_nodes', right_on = 'real_id', how= 'left')

In [330]:
## create a mask to filter out all the mutations than has been lost 
mask_mutations_lost = replace_oldbynew_nodes['new_ids'].notna()

In [332]:
## filter out mutations that has been lost 
table_mutations = tables_og.mutations[mask_mutations_lost]

In [333]:
## now replace the old nodes ids by the new nodes ids with the mapper
ids_to_replace = replace_oldbynew_nodes.dropna()['new_ids']
table_mutations.node = np.array(ids_to_replace.astype('int32'))

In [334]:
## and jsut set the sites from 0 to the length of mutation table 
table_mutations.site = np.array(range(0, len(table_mutations))).astype('int32')

In [335]:
## apply the same filter from the mutations table to the sites table 
table_sites = tables_og.sites[mask_mutations_lost]  

In [336]:
## now replace all this filter old tables in the new tree seq! 
new_tables = ts_new.dump_tables()

In [337]:
new_tables.mutations.replace_with(table_mutations)

In [338]:
new_tables.sites.replace_with(table_sites)

In [339]:
## make sure to compure mutations parents
new_tables.compute_mutation_parents()

In [340]:
## create tree seq based on tables
new_ts_plusmut = new_tables.tree_sequence()

In [ ]:
## simplify tree to get rid of all the nodes that are not related to the sample nodes (individuals that survived)
new_ts_plusmut = new_ts_plusmut.simplify()

In [341]:
## save treee with overlay neutral mutations 
new_ts_plusmut.dump('tree_output_completegenome.trees')

In [ ]:
# create a vcf file from the treeseq 
with open('test_output_tree_wmut.vcf', 'w') as file:
    # Pass the file object as the output parameter
    new_ts_plusmut.write_vcf(output=file)

## Part 5: Generate chromosomes and real positions in vcf file 

In [ ]:
awk -F'\t' 'BEGIN{OFS="\t"} $1 ~ /^#/ {print} $1 !~ /^#/ {pos=$2; 
    if (pos > 30427671 && pos <= 50125959) {pos -= 30427671; $1="2"} 
    else if (pos > 50125959 && pos <= 73585789) {pos -= 50125960; $1="3"} 
    else if (pos > 73585789 && pos <= 92170845) {pos -= 73585790; $1="4"} 
    else if (pos > 92170845 && pos <= 119146348) {pos -= 92170846; $1="5"} 
    $2=pos; print}' output_completegenome.vcf > output_completegenome_chr.vcf

## checks 
Checks consist on comparing the simulation done from vcf file with the one from tree seq 

In [344]:
# Path to the VCF file
vcf_file_path = 'test_output.vcf'

# Open the VCF file
callset_from_vcf = allel.read_vcf(vcf_file_path)

# Path to the VCF file
vcf_file_path = 'test_output_tree_wmut.vcf'
# Open the VCF file
callset_from_tree = allel.read_vcf(vcf_file_path)

In [346]:
len(callset_from_vcf['variants/POS'])

542

In [347]:
len(callset_from_tree['variants/POS'])

542

In [350]:
gtfrom_tree = allel.GenotypeArray(callset_from_tree['calldata/GT'])
gtfrom_vcf = allel.GenotypeArray(callset_from_vcf['calldata/GT'])

In [351]:
missing_fromvcf = [i for i in callset_from_tree['variants/POS'] if i not in callset_from_vcf['variants/POS']]

In [352]:
missing_fromtree = [i for i in callset_from_vcf['variants/POS'] if i not in callset_from_tree['variants/POS']]

In [353]:
len(missing_fromvcf)

0

In [354]:
missing_fromtree

[]

In [355]:
(gtfrom_tree == gtfrom_vcf).all()

True

In [22]:
# Path to the VCF file
vcf_file_path = 'two_samples.vcf'

# Open the VCF file
callset_from_vcf = allel.read_vcf(vcf_file_path)

In [23]:
callset_from_vcf['calldata/GT'].shape

(60943, 2, 2)